In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'matplotlib'

# Analysing kart data


In [ ]:
kart_data = pd.read_csv("../Data/kart_data.csv")
kart_data.describe()

In [ ]:
kart_data.head()



In [ ]:
kart_data.groupby('No.').count()['Pos'].describe()

### Selecting just karts with 100 or more races

In [ ]:
filter_karts_500races = kart_data.groupby('No.').count()['Pos'] >= 500
karts_500races = kart_data.groupby('No.').count()['Pos'][filter_karts_500races].reset_index()["No."]
karts_500races.head()

In [ ]:
filter_karts = kart_data['No.'].isin(karts_500races)
kart_data = kart_data[filter_karts]
kart_data.describe()

In [ ]:
kart_data['Pos'] = pd.to_numeric(kart_data['Pos'],errors='coerce')
kart_data.pivot(columns='No.',values='Pos').describe()

In [ ]:
# the boxplot
fig1, ax1 = plt.subplots()
data=[]
for i in range(1,20):
    _data = kart_data.loc[(kart_data['No.']==i) & (kart_data['Pos'] > 0), 'Pos']
    _data_mean = np.mean(_data)
    data.append((i,_data_mean,data))

data.sort(key=lambda x: x[1])

# ax1.boxplot(data[0][2], vert=False, meanline=True)

In [ ]:
kart_data.loc[kart_data['No.']==1]['Pos'].reset_index().head()

In [ ]:
# Fixing random state for reproducibility
np.random.seed(19680801)

# fake up some data
spread = np.random.rand(50) * 100
center = np.ones(25) * 50
flier_high = np.random.rand(10) * 100 + 100
flier_low = np.random.rand(10) * -100
data = np.concatenate((spread, center, flier_high, flier_low))

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Basic Plot')
ax1.boxplot(data)

In [ ]:

data